# Tarea 4
0226594 || Sara Carolina Gómez Delgado

### Utils

In [1007]:
import matplotlib
from sklearn.ensemble import RandomForestClassifier
import pandas
from bs4 import BeautifulSoup
import numpy as np
import scipy
import nltk 
from nltk.tokenize import TweetTokenizer
from itertools import islice
import random
import mpmath

In [1008]:
def get_texts_from_file(path_corpus, path_truth):
    tr_txt = []
    tr_y = []
    with open(path_corpus, "r",encoding="utf8") as f_corpus, open(path_truth, "r",encoding="utf8") as f_truth:
        for tuit in f_corpus:
            tr_txt += [tuit]
        for label in f_truth:
            tr_y += [label] 
    return tr_txt, tr_y

_Calculate Perplexity_

In [1009]:
def perplexity(prob, N): 
    inv = 1/prob
    return mpmath.root(inv, N)

---
<h1 style="text-align: center;">Parte 1</h1>

<h4 style="text-align: center;"><i>"Modelos de Lenguaje y Evaluación"</i></h4>

---

#### 1.1 Preprocesamiento

_<kbd>Instrucción</kbd>_

Preprocese todos los tuits de agresividad (positivos y negativos) según su intuición para construir un buen corpus para un modelo de lenguaje (e.g., solo palabras en minúscula, etc.). Agregue tokens especiales de < s > y </ s > según usted considere (e.g., al
inicio y final de cada tuit). Defina su vocabulario y enmascare con < unk > toda palabra
que no esté en su vocabulario.


_<kbd>Comentario</kbd>_

En esta sección se preprocesaron los tweets y se generó un corpus donde sólo se consideraron palabras (no emojis, no signos de puntuación) y se eligió un vocabulario tomando las palabras únicas en el training set, después se ordenaron según su frecuencia (mayor a menor) y se tomaron sólo las palabras cuya frecuencia fue mayor a 50 para formar el vocabulario final. Me parece importante resaltar, como principal conclusión, la importancia de descartar palabras que aparecen poco, ya que pueden llegar a afectar fuertemente al intentar aplicar un modelo de lenguaje.

_Leer tweets_

In [1010]:
tr_txt, tr_y = get_texts_from_file("./mex_train.txt", "./mex_train_labels.txt")

_Pre-procesar tweets_

In [1011]:
# Create TweetTokenizer instance
tokenizer = TweetTokenizer()

In [1012]:
# <s> at the beggining of a tweet and </s> at the end of this tweet
corpus_palabras = []
for doc in tr_txt:
    x = "<s>" + doc + "</s>"
    corpus_palabras += tokenizer.tokenize(x) 

In [1013]:
processed = []
for word in corpus_palabras:
    if(np.char.isalpha(word) or word == "<s>" or word == "</s>"):
            processed.append(word)
print(processed[:10])

['<s>', 'lo', 'peor', 'de', 'todo', 'es', 'que', 'no', 'me', 'dan']


In [1014]:
fdist = nltk.FreqDist(processed) # frecuencia de cada palabra
def sortFreqDict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict] #lista de pares ordenada (más frecuente a menos)
    aux.sort()
    aux.reverse()
    return aux #regresa el objeto ordenado en reversa (más frecuentes a menos frecuentes)

V = sortFreqDict(fdist)

In [1015]:
# if freq < 50, ignore it (don't let it be part of vocab)
vocab = []
for item in V:
    if item[0] > 50:
        vocab.append(item[1])
vocab[:10]


['<s>', '</s>', 'que', 'de', 'a', 'la', 'y', 'no', 'me', 'el']

In [1016]:
for i,word in enumerate(processed):
    if word not in vocab:
        processed[i] = "<unk>"
print(processed[:10])

['<s>', 'lo', '<unk>', 'de', 'todo', 'es', 'que', 'no', 'me', '<unk>']


#### 1.2 Entrenamiento (unigrama, bigrama y trigrama)

_<kbd>Instrucción</kbd>_

Entrene tres modelos de lenguaje sobre todos los tuis: $P_{unigramas}(w^n_1), P_{bigramas}(w^n_1), P{trigramas}(w^n_1).$ Para cada uno proporcione  una interfaz (función) sencilla para $P{n-grama}(w^n_1)$ y $Pn-grama(w^n_1 | w^{n-1}_{n-N+1})$. Los modelos modelos deben tener una estrategia común para lidiar con
secuencias no vistas. Puede optar por un suavizamiento Laplace o un Good-Turing
discounting. Muestre un par de ejemplos de como funciona, al menos uno con una
palabra fuera del vocabulario.


_<kbd>Comentario</kbd>_

En esta sección decidí crear dos funciones aplicando en ambas Laplace Smoothing (Add-One) para manejar el caso de secuencias no vistas.
1) Función que devuelve la probabilidad de que suceda una palabra dada un contexto.
2) Función que devuelve la probabilidad de que suceda una oración.

Como conclusion, me llamó mucho la atención el hecho de que cuando se encuentra el modelo con una palabra que no ha visto antes, no la vuelve cero (por al add-one de Laplace), si no que su probabilidad se vuelve muy pequeña.

In [1017]:
def prob_word(words, context, vocab_size): # (list)
    assert isinstance(words, list)
    assert isinstance(context, list)
    assert len(context) > 0
    if len(words) == 1: # prob = (word_freq + k) / (context_size + k*V)
        print("unigram")
        word = words[0]
        word_freq = context.count(word)
        context_size = len(context)
        prob = (word_freq+1)/(context_size+vocab_size)
        return prob
    elif len(words) == 2: # prob = (bigram_freq + k) / (a_freq + k*V)
        print("bigram")
        a,b = words
        a_freq = context.count(a)
        bigram_freq = 0
        for i in range(len(context)-1):
            if context[i]==a and context[i+1]==b:
                bigram_freq += 1
        prob = (bigram_freq+1)/(a_freq+vocab_size)
        return prob
    elif len(words) == 3: # prob = (count_trigram + k) / (count_bigram + k*V)
        print("trigram")
        count_trigram = 0
        count_bigram = 0
        for i in range(len(context)-2):
            # number of times the trigram appears
            if context[i:i+3] == words:
                count_trigram += 1
            # number of times the first two words of the trigram appear in the context (bigram)
            if context[i:i+2] == words[:2]:
                count_bigram += 1
        prob = (count_trigram+1)/(count_bigram + vocab_size)
        return prob

# unigrama

result = prob_word(["a"], processed, len(processed))
print("\t{:.10f}".format(result))

# bigrama
result = prob_word(["lo", "que"], processed, len(processed))
print("\t{:.10f}".format(result))

# trigrama
result = prob_word(["si", "no", "fuera"], processed, len(processed))
print("\t{:.10f}".format(result))

# These words don't belong to the vocabulary
result = prob_word(["hello", "dude"], processed, len(processed))
print("\t{:.10f}".format(result))



unigram
	0.0133005624
bigram
	0.0013767184
trigram
	0.0000404285
bigram
	0.0000101145


In [1018]:
def ngram_prob(n, sentence, corpus):
    assert isinstance(n, int)
    assert isinstance(sentence, list)
    assert n <= len(sentence)
    assert isinstance(corpus, list)

    freq = {}
    for i in range(len(corpus)-n+1):
        gram = tuple(corpus[i:i+n]) # n-gram (1-3)
        if gram in freq:
            freq[gram] += 1
        else:
            freq[gram] = 1
    # Laplace smoothing
    V = len(set(corpus))
    for gram in freq:
        freq[gram] += 1
    count = 0
    for val in freq.values():
        count += val
    count += V
    prob = (freq.get(tuple(sentence[-n:]),0)+1)/count
    return prob

ngram = 1
result = ngram_prob(ngram, ['de', 'todo', 'lo', 'que'], processed)
if ngram == 1: print("unigram")
elif ngram == 2: print("bigram")
elif ngram == 3: print("trigram")
else: print("n-gram")
print("\t{:.10f}".format(result))


unigram
	0.0341057935


#### 1.3 Construcción de Modelo Interpolado

_<kbd>Instrucción</kbd>_

Construya un modelo interpolado con valores $\lambda$ fijos:

$$\hat{P}(w_n|w_{n-2}w_{n-1}) = \lambda_1P(w_n|w_{n-2}w_{n-1}) + \lambda_2P(w_n|w_{n-1}) + \lambda_3P(w_n)$$

Para ello experimente con el modelo en particiones estratificadas de 80%, 10% y 10% para
entrenar (train), ajuste de parámetros (val) y prueba (test) respectivamente. Muestre como
bajan o suben las perplejidades en validación, finalmente pruebe una vez en test. Para esto puede explorar algunos valores ⃗λ y elija el mejor, i.e., [1/3, 1/3, 1/3],[.4, .4, .2],[.2, .4, .4],[.5, .4, .1]
y [.1, .4, .5].

_<kbd>Comentario</kbd>_

En esta sección dividí primero mis datos en 80% train, 10% validation y 10% test. Después, creé el modelo interpolado donde $\lambda_1P(w_n|w_{n-2}w_{n-1})$ representa un trigrama multiplicado por un valor lambda_1, \lambda_2P(w_n|w_{n-1}) representa un bigrama multiplicado por un valor de lambda_2 y \lambda_3P(w_n) representa un unigrama multiplicado por un valor de lambda_3. Los resultados son sumados. Sumé estos valores y finalmente experimenté con los diferentes valores de lambda para comparar sus perplejidades y quedarme con el valor más pequeño.

_Data Partition (80% (train), 10% validation, 10% test)_

In [1019]:
# Shuffle data
rand_data = processed.copy()
random.shuffle(rand_data)


# Divide 80% Train, 20% Test
n = len(rand_data) 
n_train = int(n * 0.8)  # entrenamiento (80%)
n_test = n - n_train  # prueba (20%)
train = rand_data[:n_train]
to_divide = rand_data[n_train:]

# Divide that 20% in two parts (10% Validation, 10% Test)
n_test = len(to_divide) 
n_val = n_test//2  # Validation (50%)
n_test = n_test - n_val  # Test(50%)
validation = to_divide[:n_val]
test = to_divide[n_val:]

print("Train: ",(len(train)*100)/len(processed))
print("Validation: ",(len(validation)*100)/len(processed))
print("Test: ",(len(test)*100)/len(processed))

Train:  79.99959542015617
Validation:  10.000202289921916
Test:  10.000202289921916


_Find Best Lambda_

In [1020]:
def best_lambda(uni, bi, tri, lambdas, N):
    assert len(lambdas) >= 1
    best_lam = lambdas[0]
    best_perplexity = 10000
    for l in lambdas:
        probability = uni*l[0] + bi*l[1] + tri*l[2]
        p = perplexity(probability, N)
        if(p < best_perplexity):
            best_perplexity = p
            best_lam = [l[0], l[1], l[2]]
    return best_lam, best_perplexity

_Validation_

In [1021]:
lambdas = [[1/3, 1/3, 1/3], [.4, .4, .2], [.2, .4, .4], [.5, .4, .1], [.1, .4, .5]]

# unigram
ngram = 1
uni = ngram_prob(ngram, validation, train)
print("Unigram: {:.10f}".format(uni))

# bigram
ngram = 2
bi = ngram_prob(ngram, validation, train)
print("Bigram: {:.10f}".format(bi))

# trigram
ngram = 3
tri = ngram_prob(ngram, validation, train)
print("Trigram: {:.10f}".format(tri))

res = best_lambda(uni,bi,tri, lambdas, len(validation))
validation_best_lambda = res[0]
validation_best_perplexity = res[1]
print("best lambda and its perplexity: ",validation_best_lambda, validation_best_perplexity)

Unigram: 0.0008304394
Bigram: 0.0000448355
Trigram: 0.0000087011
best lambda and its perplexity:  [0.5, 0.4, 0.1] 1.00078339668382


_Test_

In [1022]:
lambdas = [[1/3, 1/3, 1/3], [.4, .4, .2], [.2, .4, .4], [.5, .4, .1], [.1, .4, .5]]

# unigram
ngram = 1
uni = ngram_prob(ngram, test , train)
print("Unigram: {:.10f}".format(uni))

# bigram
ngram = 2
bi = ngram_prob(ngram, test , train)
print("Bigram: {:.10f}".format(bi))

# trigram
ngram = 3
tri = ngram_prob(ngram, test , train)
print("Trigram: {:.10f}".format(tri))

res = best_lambda(uni,bi,tri, lambdas, len(test))
test_best_lambda = res[0]
test_best_perplexity = res[1]
print("best lambda and its perplexity: ", test_best_lambda, test_best_perplexity)

Unigram: 0.0244098847
Bigram: 0.0073306058
Trigram: 0.0000087011
best lambda and its perplexity:  [0.5, 0.4, 0.1] 1.00042393362114


_Show Best Perplexity_

In [1023]:
if validation_best_perplexity < test_best_perplexity:
    print("Best Perplexity (validation): ", validation_best_perplexity)
    print("Lambdas: ", validation_best_lambda)
else:
    print("Best Perplexity (test): ", test_best_perplexity)
    print("Lambdas: ", test_best_lambda)

Best Perplexity (test):  1.00042393362114
Lambdas:  [0.5, 0.4, 0.1]


---
<h1 style="text-align: center;">Parte 2</h1>

<h4 style="text-align: center;"><i>"Generación de Texto"</i></h4>

---

Para esta parte reentrenará su modelo de lenguaje interpolado para aprender los valores λ:

$$\hat{P}(w_n|w_{n-2}w_{n-1}) = \lambda_1P(w_n|w_{n-2}w_{n-1}) + \lambda_2P(w_n|w_{n-1}) + \lambda_3P(w_n)$$

#### 2.1 Función "tuitear"

_<kbd>Instrucción</kbd>_

Haga una función "tuitear" con base en su modelo de lenguaje P̂ del último punto.
El modelo deberá poder parar automáticamente cuando genere el símbolo de terminación
de tuit al final (e.g., "</s>"), o 50 palabras. Proponga algo para que en los últimos tokens
sea más probable generar el token "</s>". Muestre al menos cinco ejemplos.

In [ ]:
def tuitear():
    pass

#### 2.2 Entrenar modelo de lenguaje AMLO

_<kbd>Instrucción</kbd>_

Use la intuición que ha ganado en esta tarea y los datos de las mañaneras para
entrenar un modelo de lenguaje AMLO. Haga una un función "dar_conferencia()". Generé
un discurso de 300 palabras y detenga al modelo de forma abrupta.

#### 2.3 Experimentando con dos frases

_<kbd>Instrucción</kbd>_

Calcule el estimado de cada uno sus modelos de lenguaje (el de tuits y el de amlo)
para las frases: "sino gano me voy a la chingada", "ya se va a acabar la corrupción".

#### 2.4 Permutaciones

_<kbd>Instrucción</kbd>_

Para cada oración del punto anterior, haga todas las permutaciones posibles.
Calcule su probabilidad a cada nueva frase y muestre el top 3 mas probable y el top 3
menos probable (para ambos modelos de lenguaje). Proponga una frase más y haga lo
mismo.